# Trying to understand Trevor's interfaces

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [ ]:
figW, figH = plt.rcParams['figure.figsize']
plt.rcParams['figure.figsize'] = [2.5 * figW, 1.75 * figH]

## Parabolic energy wells

In [ ]:
def f_gen(C, a, C0, f0):
    return (1./(4*a)) * (C - C0)**2 + f0

In [ ]:
def dfdC_gen(C, a, C0):
    return (1./(2*a)) * (C - C0)

In [ ]:
def fS(C):
    return f_gen(C, a=aS, C0=C0S, f0=f0S)

In [ ]:
def dfSdC(C):
    return dfdC_gen(C, a=aS, C0=C0S)

In [ ]:
def fL(C):
    return f_gen(C, a=aL, C0=C0L, f0=f0L)

In [ ]:
def dfLdC(C):
    return dfdC_gen(C, a=aL, C0=C0L)

## Equilibrium

Determined by common tangent:
$$\left.\frac{\partial f_S}{\partial C_S}\right|_{C_S^{eq}} = \left.\frac{\partial f_L}{\partial C_L}\right|_{C_L^{eq}}$$
$$\left.f_L\right|_{C_L^{eq}} = \left.f_S\right|_{C_S^{eq}} + \left(C_L^{eq} - C_S^{eq}\right)\left.\frac{\partial f_S}{\partial C_S}\right|_{C_S^{eq}} $$

In [ ]:
def equilibrium(C):
    return [dfSdC(C[0]) - dfLdC(C[1]),
            fL(C[1]) - fS(C[0]) - (C[1] - C[0]) * dfSdC(C[0])]

## Fictitious compositions

Determined by parallel tangent
$$\left.\frac{\partial f_S}{\partial C_S}\right|_{C_S} = \left.\frac{\partial f_L}{\partial C_L}\right|_{C_L}$$
and relation between system composition and fictitious compositions
$$C = h(\phi) C_S + [1 - h(\phi)] C_L$$

### Numerical solution

In [ ]:
def fictitious(x, *args):
    CS, CL = x
    C, phi = args
    return [dfSdC(CS) - dfLdC(CL),
            C - h(phi) * CS - (1 - h(phi)) * CL]

### Analytical solution

In [ ]:
def CL(C, phi):
    return (aL * C + h(phi) * (aS * C0L - aL * C0S))/(aS * h(phi) + aL * (1 - h(phi)))

In [ ]:
def CS(C, phi):
    return (aS * C - (1 - h(phi)) * (aS * C0L - aL * C0S))/(aS * h(phi) + aL * (1 - h(phi)))

## Model Parameters

In [ ]:
d0 = 0.2   # width of the interface
w0 = 10.   # barrier height between energy wells
dens = 256 # number of points in np.linspace calls

## Plot free energy and common tangent

In [ ]:
aL = 0.001
C0L = 0.775
f0L = 0.

aS = 0.01
C0S = 0.375
f0S = 0.125

In [ ]:
CSeq, CLeq = fsolve(func=equilibrium, x0=(0., 0.8))
print (CSeq, CLeq)

In [ ]:
C = np.linspace(0, 1, dens)

In [ ]:
d = d0
w = w0
plt.plot(C, fS(C), label="S")
plt.plot(C, fL(C), label="L")
plt.plot([CSeq, CLeq], [fS(CSeq), fL(CLeq)])
plt.ylim(0, 2)
plt.xlim(0, 1)
plt.legend(loc="best")

### Interpolators

In [ ]:
def h(phi):
    return phi**3 * (6 * phi**2 - 15 * phi + 10)

In [ ]:
def g(phi):
    return phi**2 * (1 - phi)**2

### System free energy

In [ ]:
def f(xS, xL, phi):
    return h(phi) * fS(xS) + (1 - h(phi)) * fL(xL) + w * g(phi)

## Plot hypothetical interfaces

In [ ]:
x = np.linspace(-1, 1, dens)

### Equilibrium

In [ ]:
d = d0
w = w0
CS0, CL0 = CSeq, CLeq
phi = 0.5 * (1 + np.tanh(x/(2*d)))
# plt.plot(x, phi)
C = h(phi) * CS0 + (1 - h(phi)) * CL0
CSfic, CLfic = np.array([fsolve(func=fictitious, x0=(0.5, 0.5), args=(Cval, phival)) for Cval, phival in zip(C, phi)]).T
plt.subplot(1, 2, 1)
plt.plot(x, C)
plt.plot(x, CSfic, linestyle="--")
plt.plot(x, CLfic, linestyle="--")
#plt.plot(x, CS(C, phi), linestyle=":")
#plt.plot(x, CL(C, phi), linestyle=":")
plt.subplot(1, 2, 2)
plt.plot(x, f(CSfic, CLfic, phi))

### Slightly depleted liquid

In [ ]:
d = d0
w = w0
C0 = 0.79961
phi0 = 0.
CL0 = CL(C=C0, phi=phi0)
CS0 = CS(C=C0, phi=phi0)
print (CS0, CL0)

In [ ]:
phi = 0.5 * (1 + np.tanh(x/(2*d)))
#plt.plot(x, phi)
C = h(phi) * CS0 + (1 - h(phi)) * CL0
CSfic, CLfic = np.array([fsolve(func=fictitious, x0=(0.5, 0.5), args=(Cval, phival)) for Cval, phival in zip(C, phi)]).T
plt.subplot(1, 2, 1)
plt.plot(x, C)
plt.plot(x, CSfic, linestyle="--")
plt.plot(x, CLfic, linestyle="--")
#plt.plot(x, CS(C, phi), linestyle=":")
#plt.plot(x, CL(C, phi), linestyle=":")
plt.subplot(1, 2, 2)
plt.plot(x, f(CSfic, CLfic, phi))

### Slightly enriched liquid

In [ ]:
d = d0
w = w0
C0 = 0.79999
phi0 = 0.
CL0 = CL(C=C0, phi=phi0)
CS0 = CS(C=C0, phi=phi0)
print (CS0, CL0)

In [ ]:
phi = 0.5 * (1 + np.tanh(x/(2*d)))
#plt.plot(x, phi)
C = h(phi) * CS0 + (1 - h(phi)) * CL0
CSfic, CLfic = np.array([fsolve(func=fictitious, x0=(0.5, 0.5), args=(Cval, phival)) for Cval, phival in zip(C, phi)]).T
plt.subplot(1, 2, 1)
plt.plot(x, C)
plt.plot(x, CSfic, linestyle="--")
plt.plot(x, CLfic, linestyle="--")
#plt.plot(x, CS(C, phi), linestyle=":")
#plt.plot(x, CL(C, phi), linestyle=":")
plt.subplot(1, 2, 2)
plt.plot(x, f(CSfic, CLfic, phi))

### Very depleted liquid

In [ ]:
d = d0
w = w0
C0 = 0.7
phi0 = 0.
CL0 = CL(C=C0, phi=phi0)
CS0 = CS(C=C0, phi=phi0)
print (CS0, CL0)

In [ ]:
phi = 0.5 * (1 + np.tanh(x/(2*d)))
#plt.plot(x, phi)
C = h(phi) * CS0 + (1 - h(phi)) * CL0
CSfic, CLfic = np.array([fsolve(func=fictitious, x0=(0.5, 0.5), args=(Cval, phival)) for Cval, phival in zip(C, phi)]).T
plt.subplot(1, 2, 1)
plt.plot(x, C)
plt.plot(x, CSfic, linestyle="--")
plt.plot(x, CLfic, linestyle="--")
#plt.plot(x, CS(C, phi), linestyle=":")
#plt.plot(x, CL(C, phi), linestyle=":")
plt.subplot(1, 2, 2)
plt.plot(x, f(CSfic, CLfic, phi))

### Very enriched solid

In [ ]:
d = d0
w = w0
C0 = 0.9
phi0 = 1.
CL0 = CL(C=C0, phi=phi0)
CS0 = CS(C=C0, phi=phi0)
print (CS0, CL0)

In [ ]:
phi = 0.5 * (1 + np.tanh(x/(2*d)))
# plt.plot(x, phi)
C = h(phi) * CS0 + (1 - h(phi)) * CL0
CSfic, CLfic = np.array([fsolve(func=fictitious, x0=(0.5, 0.5), args=(Cval, phival)) for Cval, phival in zip(C, phi)]).T
plt.subplot(1, 2, 1)
plt.plot(x, C)
plt.plot(x, CSfic, linestyle="--")
plt.plot(x, CLfic, linestyle="--")
#plt.plot(x, CS(C, phi), linestyle=":")
#plt.plot(x, CL(C, phi), linestyle=":")
plt.subplot(1, 2, 2)
plt.plot(x, f(CSfic, CLfic, phi))